In [5]:
%pip install pandas
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pyspark
from pyspark.sql import SparkSession
from pymongo import MongoClient
import collections
import pandas as pd

# Crea una SparkSession
spark = SparkSession.builder \
    .appName("MongoSparkConnector2") \
    .getOrCreate()

# Conecta a MongoDB usando pymongo
client = MongoClient("mongodb://172.17.0.3:27017/")
db = client['docker']
collection = db['perfilesmongo']

# Extrae los datos desde MongoDB
mongo_data = list(collection.find())

#print(mongo_data)

# Convierte los datos a un DataFrame de pandas
pdf = pd.DataFrame(mongo_data)

# Elimina la columna '_id' si es necesario, ya que no es serializable por defecto en Spark
if '_id' in pdf.columns:
    pdf = pdf.drop(columns=['_id'])

# Convierte el DataFrame de pandas a un DataFrame de Spark
df = spark.createDataFrame(pdf)

# Convertir el DataFrame a un RDD
rdd = df.rdd
#df.show()

rdd = df.select("edad", "numcontactos").rdd

rdd_filtrado = rdd.map(lambda x: (x['edad'], x['numcontactos']))


# Calcular el número de edades únicas
edades_unicas = rdd_filtrado.map(lambda x: x[0]).distinct().count()
print(f"Número de edades únicas: {edades_unicas}")

# Calcular el promedio de numcontactos
suma_numcontactos, cuenta = rdd_filtrado.map(lambda x: (x[1], 1)) \
                                        .reduce(lambda a, b: (a[0] + b[0], a[1] + b[1]))

promedio_numcontactos = suma_numcontactos / cuenta
print(f"Promedio de numcontactos: {promedio_numcontactos}")

Número de edades únicas: 21
Promedio de numcontactos: 471.8160621761658
